In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier


#초기화 --장고 init에 있다
cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

In [2]:
# emotion_keyword_to_vec를 만든다. 인코딩용. {‘감정’, 인코딩결과} 형태

# 8가지 감정 키워드
emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
emotion_df['emotion_encoding'] = pd.factorize(emotion_df['emotion_keyword'])[0]  # pd.factorize(df['Col'])[0]

#print(emotion_df.values)
emotion_keyword_to_vec = {} #딕셔너리로 저장하면 편하겠지?
for keyword,encoding in emotion_df.values:
    #print(keyword,encoding)
    emotion_keyword_to_vec[keyword] = encoding

emotion_df

,emotion_keyword,emotion_encoding
0,anger,0
1,contempt,1
2,disgust,2
3,fear,3
4,happiness,4
5,neutral,5
6,sadness,6
7,surprise,7


In [ ]:
emotion_keyword_to_vec

In [ ]:
del emotion_df  #불필요한 메모리 제거

In [3]:
# order에 있는 데이터 가져오자
# doc_df를 만들었다

docs = firestore.client().collection(u'order').stream()

doc_list = []
for doc in docs:
    doc_list_row = []
    
    # 메뉴 - 레이블
    #doc_list_row.append(doc.get('orderToString'))
    # 데이터가 거의 없는 문제로 임시로 name으로 모델 학습한다
    doc_list_row.append(doc.get('items')[0].get('name'))

    # 날씨
#    weather_key_list = []
    for key in doc.get('weather').keys():
        doc_list_row.append(doc.get('weather').get(key))
    
    # 감정
    emotion_key_list = ''
    for key in doc.get('emotion').keys():
        emotion_key_list += str(emotion_keyword_to_vec[key])  # 감정 데이터 전처리 str to str 
    doc_list_row.append(emotion_key_list)
    
    # 날짜
    #print(type(doc.get('today')))
    #print(doc.get('today').split(' '))
    list_date = doc.get('today').split(' ')
    doc_list_row.append(list_date[0])
    doc_list_row.append(list_date[1])
    
    doc_list.append(doc_list_row)

doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion','date','time'])
doc_df['emotion'] = doc_df['emotion'].astype(int) # 감정 데이터 전처리 str to int
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,date,time
0,카푸치노,80.0,300.00,1.0,54,2019-10-04,01:40:57
1,아메리카노,80.0,300.00,1.0,54,2019-10-04,01:41:20
2,사케타로ICE,80.0,300.00,1.0,54,2019-10-04,01:42:54
3,자몽에이드,80.0,300.00,1.0,54,2019-10-04,15:53:28
4,레몬요거트스무디,80.0,300.00,1.0,54,2019-10-04,20:25:27
5,더블카카오,80.0,300.00,1.0,54,2019-10-04,13:53:59
6,브라질드립커피,80.0,300.00,1.0,54,2019-10-04,01:38:57
7,바나나프레시,80.0,300.00,1.0,54,2019-10-04,01:41:09
8,밀크티,80.0,300.00,1.0,54,2019-10-04,01:37:03
9,플레인요거트스무디,80.0,300.00,1.0,54,2019-10-04,01:45:18


In [4]:
# DataFrame 전처리
# 날 수 세는 함수
def date_to_VIP(date):
    year,month,day = date.split('-')
    year = int(year)
    month = int(month)
    day = int(day)
    preYear = year-1
    num_of_days = preYear*365 + (preYear/4 - preYear/100 + preYear/400)
    
    monthArray = [31,28,31,30,31,30,31,31,30,31,30,31]
    for m in range(month):
        num_of_days += monthArray[m]
    
    cond = (month>=3 and (year%4==0 and year%100!=0 or year%400==0))
    if cond == True:
        num_of_days += 1
    
    num_of_days += day
    
    #return int(num_of_days)
    #print(num_of_days,num_of_days % 7 - 3)
    
    return int(num_of_days)
     
    #dayArray = ['일','월','화','수','목','금','토']
    
    #print(dayArray[int(dayOfWeek)], int(num_of_days)%7 ,int(int(num_of_days-3)%7))

#    dayOfWeek = num_of_days % 7 - 3
#    if int(dayOfWeek) ==0 or int(dayOfWeek)==6:
#        return 0 # 주말
#    else:
#        return 1 # 평일
    
    #return int(num_of_days)#dayArray[int(dayOfWeek)] # 연,월,일 전체 날 수

In [5]:
# 날 수를 평일/휴일로 변환하는 함수
def num_of_days_to_work_or_free(num_of_days):
    
    dayOfWeek = num_of_days % 7 - 3
    if int(dayOfWeek) ==0 or int(dayOfWeek)==6:
        return 0 # 주말
    else:
        return 1 # 평일


In [6]:
# 아침/점심/저녁 나누는 함수
def time_to_3(time):
    result = -1
    time = int(time.split(':')[0])
    if time < 11:
        result = 0 #아침은 0
    elif time < 16: #오후는 1
        result = 1
    else:
        result = 2 #저녁은 2
    
    return result

In [7]:
# 메뉴 이름
doc_df['item_name'], item_name_restore = pd.factorize(doc_df['item_name']) # encoding 메뉴이름
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,date,time
0,0,80.0,300.00,1.0,54,2019-10-04,01:40:57
1,1,80.0,300.00,1.0,54,2019-10-04,01:41:20
2,2,80.0,300.00,1.0,54,2019-10-04,01:42:54
3,3,80.0,300.00,1.0,54,2019-10-04,15:53:28
4,4,80.0,300.00,1.0,54,2019-10-04,20:25:27
5,5,80.0,300.00,1.0,54,2019-10-04,13:53:59
6,6,80.0,300.00,1.0,54,2019-10-04,01:38:57
7,7,80.0,300.00,1.0,54,2019-10-04,01:41:09
8,8,80.0,300.00,1.0,54,2019-10-04,01:37:03
9,9,80.0,300.00,1.0,54,2019-10-04,01:45:18


In [8]:
# date > 날 수
doc_df['num_of_days'] = doc_df['date'].apply(lambda x: date_to_VIP(x))
doc_df
# date > 요일 > 주중/주말

,item_name,w_humidity,w_temp,w_speed,emotion,date,time,num_of_days
0,0,80.0,300.00,1.0,54,2019-10-04,01:40:57,737367
1,1,80.0,300.00,1.0,54,2019-10-04,01:41:20,737367
2,2,80.0,300.00,1.0,54,2019-10-04,01:42:54,737367
3,3,80.0,300.00,1.0,54,2019-10-04,15:53:28,737367
4,4,80.0,300.00,1.0,54,2019-10-04,20:25:27,737367
5,5,80.0,300.00,1.0,54,2019-10-04,13:53:59,737367
6,6,80.0,300.00,1.0,54,2019-10-04,01:38:57,737367
7,7,80.0,300.00,1.0,54,2019-10-04,01:41:09,737367
8,8,80.0,300.00,1.0,54,2019-10-04,01:37:03,737367
9,9,80.0,300.00,1.0,54,2019-10-04,01:45:18,737367


In [9]:
# 0은 주말, 1은 평일
doc_df['work_or_free'] = doc_df['num_of_days'].apply(lambda x: num_of_days_to_work_or_free(x))
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,date,time,num_of_days,work_or_free
0,0,80.0,300.00,1.0,54,2019-10-04,01:40:57,737367,1
1,1,80.0,300.00,1.0,54,2019-10-04,01:41:20,737367,1
2,2,80.0,300.00,1.0,54,2019-10-04,01:42:54,737367,1
3,3,80.0,300.00,1.0,54,2019-10-04,15:53:28,737367,1
4,4,80.0,300.00,1.0,54,2019-10-04,20:25:27,737367,1
5,5,80.0,300.00,1.0,54,2019-10-04,13:53:59,737367,1
6,6,80.0,300.00,1.0,54,2019-10-04,01:38:57,737367,1
7,7,80.0,300.00,1.0,54,2019-10-04,01:41:09,737367,1
8,8,80.0,300.00,1.0,54,2019-10-04,01:37:03,737367,1
9,9,80.0,300.00,1.0,54,2019-10-04,01:45:18,737367,1


In [10]:
# time > 아침(~11)/오후(~16)/저녁(16~)
doc_df['time_to_3'] = doc_df['time'].apply(lambda x: time_to_3(x))
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,date,time,num_of_days,work_or_free,time_to_3
0,0,80.0,300.00,1.0,54,2019-10-04,01:40:57,737367,1,0
1,1,80.0,300.00,1.0,54,2019-10-04,01:41:20,737367,1,0
2,2,80.0,300.00,1.0,54,2019-10-04,01:42:54,737367,1,0
3,3,80.0,300.00,1.0,54,2019-10-04,15:53:28,737367,1,1
4,4,80.0,300.00,1.0,54,2019-10-04,20:25:27,737367,1,2
5,5,80.0,300.00,1.0,54,2019-10-04,13:53:59,737367,1,1
6,6,80.0,300.00,1.0,54,2019-10-04,01:38:57,737367,1,0
7,7,80.0,300.00,1.0,54,2019-10-04,01:41:09,737367,1,0
8,8,80.0,300.00,1.0,54,2019-10-04,01:37:03,737367,1,0
9,9,80.0,300.00,1.0,54,2019-10-04,01:45:18,737367,1,0


In [ ]:
#doc_df

In [11]:
# 그리고 필요없는 date, time은 drop한다
doc_df.drop(['date','time'],axis=1,inplace=True)
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,num_of_days,work_or_free,time_to_3
0,0,80.0,300.00,1.0,54,737367,1,0
1,1,80.0,300.00,1.0,54,737367,1,0
2,2,80.0,300.00,1.0,54,737367,1,0
3,3,80.0,300.00,1.0,54,737367,1,1
4,4,80.0,300.00,1.0,54,737367,1,2
5,5,80.0,300.00,1.0,54,737367,1,1
6,6,80.0,300.00,1.0,54,737367,1,0
7,7,80.0,300.00,1.0,54,737367,1,0
8,8,80.0,300.00,1.0,54,737367,1,0
9,9,80.0,300.00,1.0,54,737367,1,0


In [12]:
# 날 수로 오름차 정렬한다
doc_df.sort_values(by=['num_of_days'], inplace=True)
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,num_of_days,work_or_free,time_to_3
2220,36,80.0,300.00,1.0,54,737366,1,2
2076,6,80.0,300.00,1.0,54,737366,1,2
325,28,80.0,300.00,1.0,54,737366,1,1
2716,4,80.0,300.00,1.0,54,737366,1,2
323,11,80.0,300.00,1.0,54,737366,1,2
2400,20,80.0,300.00,1.0,54,737366,1,2
2079,5,80.0,300.00,1.0,54,737366,1,2
2719,26,80.0,300.00,1.0,54,737366,1,2
2398,31,80.0,300.00,1.0,54,737366,1,1
2721,26,80.0,300.00,1.0,54,737366,1,2


In [13]:
# 날 수 제거한다
doc_df.drop('num_of_days',axis=1, inplace=True)
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,work_or_free,time_to_3
2220,36,80.0,300.00,1.0,54,1,2
2076,6,80.0,300.00,1.0,54,1,2
325,28,80.0,300.00,1.0,54,1,1
2716,4,80.0,300.00,1.0,54,1,2
323,11,80.0,300.00,1.0,54,1,2
2400,20,80.0,300.00,1.0,54,1,2
2079,5,80.0,300.00,1.0,54,1,2
2719,26,80.0,300.00,1.0,54,1,2
2398,31,80.0,300.00,1.0,54,1,1
2721,26,80.0,300.00,1.0,54,1,2


In [14]:
# 특징, 레이블 index 리스트 형태로 저장
features = doc_df.columns.tolist()
# python list에서 remove(값)으로 요소 삭제할 수 있다
features.remove('item_name') # 리스트에 반영된다
label = ['item_name']
features

['w_humidity', 'w_temp', 'w_speed', 'emotion', 'work_or_free', 'time_to_3']

In [15]:
test_data_num = int(doc_df.shape[0] * 0.2) # 검증 데이터 개수

In [16]:
# 학습 데이터, 검증 데이터 분리
eval_df = doc_df[-test_data_num:]
train_df = doc_df[:-test_data_num]

In [17]:
train_df['item_name'].value_counts()

28    71
31    69
22    69
4     69
0     68
7     68
20    68
30    68
32    68
24    67
1     67
21    67
29    67
17    67
23    66
9     66
25    66
8     65
10    65
11    65
6     65
34    65
2     65
19    64
12    64
15    64
13    64
33    63
18    63
16    63
26    62
27    62
14    62
5     62
36    59
3     58
35    57
Name: item_name, dtype: int64

In [18]:
# 학습 데이터 오버 샘플링
from imblearn.over_sampling import SMOTE

print('오버샘플링 전', train_df[features].shape) # 오버샘플링 전
smote = SMOTE()

smote_x, smote_y = smote.fit_sample(train_df[features], train_df[label])

print('오버샘플링 후', smote_x.shape) # 오버샘플링 후
print(pd.Series(smote_y).value_counts()) # 가장 많은 개수에 맞춰서 증폭된다

오버샘플링 전 (2408, 6)
오버샘플링 후 (2627, 6)
35    71
34    71
30    71
28    71
26    71
24    71
22    71
20    71
18    71
16    71
14    71
12    71
10    71
8     71
6     71
4     71
2     71
32    71
36    71
33    71
1     71
31    71
29    71
27    71
25    71
23    71
21    71
19    71
17    71
15    71
13    71
11    71
9     71
7     71
5     71
3     71
0     71
dtype: int64


C:\Users\sisi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion,work_or_free,time_to_3
2220,36,80.0,300.00,1.0,54,1,2
2076,6,80.0,300.00,1.0,54,1,2
325,28,80.0,300.00,1.0,54,1,1
2716,4,80.0,300.00,1.0,54,1,2
323,11,80.0,300.00,1.0,54,1,2
2400,20,80.0,300.00,1.0,54,1,2
2079,5,80.0,300.00,1.0,54,1,2
2719,26,80.0,300.00,1.0,54,1,2
2398,31,80.0,300.00,1.0,54,1,1
2721,26,80.0,300.00,1.0,54,1,2


In [21]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimator=400, learning_rate=0.1, max_depth=3)
evals = [(eval_df[features], eval_df[label])]
xgb_wrapper.fit(train_df[features], train_df[label], early_stopping_rounds=100, eval_metric='mlogloss', eval_set=evals, verbose=True)
#pred = xgb_wrapper.predict(test_df[features])
#print(item_name_restore[pred]) # encoding to 원래 아이템 이름

[0]	validation_0-mlogloss:3.61274
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[1]	validation_0-mlogloss:3.6145
[2]	validation_0-mlogloss:3.61619
[3]	validation_0-mlogloss:3.61782
[4]	validation_0-mlogloss:3.61939
[5]	validation_0-mlogloss:3.6209
[6]	validation_0-mlogloss:3.62235
[7]	validation_0-mlogloss:3.62373
[8]	validation_0-mlogloss:3.62507
[9]	validation_0-mlogloss:3.62634
[10]	validation_0-mlogloss:3.62757
[11]	validation_0-mlogloss:3.62874
[12]	validation_0-mlogloss:3.62987
[13]	validation_0-mlogloss:3.63094
[14]	validation_0-mlogloss:3.63197
[15]	validation_0-mlogloss:3.63296
[16]	validation_0-mlogloss:3.6339
[17]	validation_0-mlogloss:3.6348
[18]	validation_0-mlogloss:3.63566
[19]	validation_0-mlogloss:3.63649
[20]	validation_0-mlogloss:3.63727
[21]	validation_0-mlogloss:3.63802
[22]	validation_0-mlogloss:3.63874
[23]	validation_0-mlogloss:3.63943
[24]	validation_0-mlogloss:3.64008
[25]	validation_0-mlogloss:3.64069
[26]	validation_0-mlogloss:3.64129

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimator=400,
              n_estimators=100, n_jobs=1, nthread=None,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [24]:
import pickle
# 학습한 모델을 저장한다
pickle.dump(xgb_wrapper, open('xgb_baseline.pkl', 'wb'))
pickle.dump(item_name_restore, open('item_name.pkl', 'wb'))

In [ ]:
-끝-

In [23]:
import json   
    # 학습한 모델을 저장한다
keys = ('model','item_name_restore')
values = (xgb_wrapper, item_name_restore)
save_dict = dict(zip(keys, values))
json.dumps(save_dict, ensure_ascii=False)

with open('xgb_baseline.pkl', 'wb') as f:
    for e in save_dict:
        pickle.dump(e, f)


TypeError: Object of type 'XGBClassifier' is not JSON serializable

In [ ]:
# XGB 모델이 입력할 형태로 변환한다 --이건 파이썬 래퍼. 나는 사이킷런 래퍼 쓸거다

#import xgboost as xgb
#
#X_trn = train_df.as_matrix(columns=features)
#Y_trn = train_df.as_matrix(columns=label)
#dm_trn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)
#
#X_test = test_df.as_matrix(columns=features)
#Y_test = test_df.as_matrix(columns=label)
#dm_test = xgb.DMatrix(X_test, label=Y_test, feature_names=features)

In [ ]:
# (임시) 모델 파라미터
# XGBoost 모델 parameter를 설정한다.
#param = {
#    'booster': 'gbtree',
#    'max_depth': 8,
#    'nthread': 4,
#    'num_class': 1,#len(prods), # 제품변수 개수
#    'objective': 'multi:softprob',
#    'silent': 1,
#    'eval_metric': 'mlogloss',
#    'eta': 0.1,
#    'min_child_weight': 10,
#    'colsample_bytree': 0.8,
#    'colsample_bylevel': 0.9,
#    'seed': 2018,
#    }

In [ ]:
# xgb 모델을 훈련 데이터로 학습한다
#watch_list = [(dm_trn, 'train'),(dm_test, 'test')]
#model = xgb.train(param, dm_trn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)